# Assignment 1

## Part 1

**a: In your opinion, what were the most important turning points in the history of deep learning?**

There have been many important turning points in the history of deep learning. From the development of the first artificial neuron in the 1940s to today, a lot has happened. 
To me personally, the most important turning points have been the advancements in visual recognition. 
This is a key moment where deep learning began to show a clear advantage over traditional machine learning.

**b: Explain the ADAM optimizer.**

The typical optimizer uses some form of gradient descent to find a minimum loss. Problems occur when there are local minima along this path. 
The algorithm might get stuck and never optimize for minimal loss. Momentum is a way to reduce this problem. The momentum algorithm remembers the direction of previous iterations and pushes in that direction. 
Another technique that accelerates convergence is by adjusting the learning rate. 
This is known as the adaptive gradient algorithm, where the learning rate becomes smaller over time as it's divided by the sum of all previous gradients.

The ADAM optimizer combines both of these features, making it one of the fastest optimizers to converge and, therefore, quite convenient. However, studies have shown that standard SGD with momentum can perform better overall.

**c: Assume data input is a single 30x40 pixel image. First layer is a convolutional layer with 5 filters, with kernel size 3x2, step size (1,1) and padding='valid'. What are the output dimensions?**

Since the padding is valid, it means that we wont add a padding around the image. The step size is (1,1), so the kernel will only move one step at the time. We have 5 filters, whih means that there will be 5 dimensions. 
$$
  \text{Output Height} = \frac{\text{Input Height} - \text{Kernel height}}{\text{Stide Height}} + 1
$$

$$
  \text{Output Width} = \frac{\text{Input Width} - \text{Kernel Width}}{\text{Stide Width}} + 1
$$

Following these equations, the output dimensions will be 28x39x5, since we have no padding.


**d: Assuming ReLU activations and offsets, and that the last layer is softmax, how many parameters does this network have:**

<img src="data\diagram.png" alt="image" width="700" height="400">

The inputer layer is 5 and we have 3 hidden layers of size 5 and an output layer of 3. The number of parameters will be:

$$
  \text{Parameters} = (5 \cdot 5 + 5) + (5 \cdot 5 + 5) + (5 \cdot 5 + 5) + (5 \cdot 3 + 3) = 3 \cdot 30 + 18 = 108
$$

**e: For a given minibatch, the targets are [1,4, 5, 8] and the network output is [0.1,4.4,0.2,10]. If the loss function is "torch.nn.HuberLoss(reduction='mean', delta=1.0)", what is the loss for this minibatch?**

For a batch of size $N$, the unreduced loss can be described as:

$$
L=\left\{l_1, \ldots, l_N\right\}^T
$$

with

$$
l_n= \begin{cases}\frac{1}{2}\left(x_n-y_n\right)^2, & \text { if }\left|x_n-y_n\right|<\delta \\ \delta\left(\left|x_n-y_n\right|-\frac{1}{2}\delta\right), & \text { otherwise }\end{cases}
$$


If reduction is not none, then:

$$
\ell(x, y)= \begin{cases}\operatorname{mean}(L), & \text { if reduction }=\text { 'mean' } \\ \operatorname{sum}(L), & \text { if reduction }=\text { 'sum' }\end{cases}
$$

This will give the result of:

$$
L = \left\{\frac{1}{2}(1-0.1)^2,\frac{1}{2}(4-4.4)^2, \delta\left(\left|5-0.2\right|-\frac{1}{2}\delta\right), \delta\left(\left|8-10\right|-\frac{1}{2}\delta\right)\right\}^T
$$

$$
L = \left\{0.405, 0.08, 4.3, 1.5\right\}^T
$$

$$
\ell(x, y)=\frac{0.405 + 0.08 + 4.3 + 1.5}{4} = \frac{6.285}{4} = 1.57125
$$

The loss of the minibatch will be 1.57125


# Part 2: Writing a PyTorch dataset

In [15]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import torch
import matplotlib.pyplot as plt

In [13]:
class InsectsDataset(Dataset):
    def __init__(self, csv_file, image_folder, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with csv and image folder.
            csv_file (string): Path to the csv file with filenames and species.
            image_folder (string): Directory with images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.image_folder = image_folder
        self.csv_path =  os.path.join(self.root_dir, csv_file)
        self.insects_df = pd.read_csv(self.csv_path)
        self.transform = transform
        
        # Create a mapping from species name to integer labels
        self.species_to_label = {species: idx for idx, species in enumerate(self.insects_df['species'].unique())}

    def __len__(self):
        return len(self.insects_df)
    
    def __getitem__(self, idx):
        # Get the image file path
        img_name = os.path.join(self.root_dir, self.image_folder, self.insects_df.iloc[idx, 2])  #'filename' is the third column
        image = Image.open(img_name)
        
        # Get the species label
        species = self.insects_df.iloc[idx, 1]  # assuming 'species' is the second column
        label = self.species_to_label[species]
        
        # Apply transformations if needed
        if self.transform:
            image = self.transform(image)
        
        return image, label

transform = transforms.Compose([
    transforms.Resize((520, 520)), # Resizing the image to 520x520
    transforms.ToTensor()
])

dataset = InsectsDataset(csv_file='insects.csv',image_folder = "Insects", root_dir='data/', transform=transform)
print(dataset.species_to_label)

{'Andrena fulva': 0, 'Panurgus banksianus': 1, 'Lasioglossum punctatissimum': 2}


In [14]:
batch_size = 4

# Set up the dataset.
dataset = dataset

# Set up the dataset.
trainloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=0)

# get some images
dataiter = iter(trainloader)
images, labels = next(dataiter)

'''
for i in range(2): #Run through 5 batches
    images, labels = next(dataiter)
    for image, label in zip(images,labels): # Run through all samples in a batch
        plt.figure()
        plt.imshow(np.transpose(image.numpy(), (1, 2, 0)))
        plt.title(label)
'''
Works = True

# Part 3: